**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Jul 29 05:43:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
 16084207_9730473_5213426836.pdf
 1751.pdf
 1752.pdf
 1753.pdf
 1756.pdf
 1799.pdf
 1801.pdf
 1888.pdf
 1920.pdf
 1928.pdf
'1930 (1).pdf'
 1930.pdf
 1931.pdf
 2017188227.pdf
 2103.pdf
 2129.pdf
 292.pdf
 5220.pdf
'52 CONIC SECTION PART 3 of 8.pdf'
'53 CONIC SECTION PART 4 of 8.pdf'
'54 CONIC SECTION PART 5 of 8.pdf'
'55 CONIC SECTION PART 6 of 8.pdf'
'56 CONIC SECTION PART 7 of 8.pdf'
'57 CONIC SECTION PART 8 of 8.pdf'
'6072296 (1).pdf'
 6072296.pdf
 alcohol.

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 14086 (delta 0), reused 1 (delta 0), pack-reused 14078
Receiving objects: 100% (14086/14086), 12.78 MiB | 7.34 MiB/s, done.
Resolving deltas: 100% (9578/9578), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/Project/Single_Object_Detection_Using_YOLO"

mkdir: cannot create directory ‘/mydrive/Project/Single_Object_Detection_Using_YOLO’: File exists


In [ ]:
!echo "BMW_car" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/Project/Single_Object_Detection_Using_YOLO' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-07-29 05:47:06--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   307KB/s    in 8m 10s  

2020-07-29 05:55:18 (324 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/Project/Single_Object_Detection_Using_YOLO/BMW_Train_dataset.zip -d data/obj

Archive:  /mydrive/Project/Single_Object_Detection_Using_YOLO/BMW_Train_dataset.zip
  inflating: data/obj/2012-bmw-3-series_100380965_l.jpeg  
  inflating: data/obj/2012-bmw-3-series_100380965_l.txt  
  inflating: data/obj/2012-BMW-3-series-front-view-6.jpeg  
  inflating: data/obj/2012-BMW-3-series-front-view-6.txt  
  inflating: data/obj/67163627.jpeg  
  inflating: data/obj/67163627.txt   
  inflating: data/obj/149105744.jpg  
  inflating: data/obj/149105744.txt  
  inflating: data/obj/149106035.thumb.jpg  
  inflating: data/obj/149106035.thumb.txt  
  inflating: data/obj/149106052.jpg  
  inflating: data/obj/149106052.txt  
  inflating: data/obj/149106128.jpg  
  inflating: data/obj/149106128.txt  
  inflating: data/obj/149106309.thumb.jpg  
  inflating: data/obj/149106309.thumb.txt  
  inflating: data/obj/149106353.jpg  
  inflating: data/obj/149106353.txt  
  inflating: data/obj/149106456.jpg  
  inflating: data/obj/149106456.txt  
  inflating: data/obj/149106459.jpg  
  inflatin

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)


15 0.436131 0.527174 0.799270 0.815217

['15', '0.436131', '0.527174', '0.799270', '0.815217']
0 data/obj/149106905.thumb.txt
0 0.436131 0.527174 0.799270 0.815217
15 0.492708 0.518056 0.885417 0.608333

['15', '0.492708', '0.518056', '0.885417', '0.608333']
1 data/obj/150084171.txt
0 0.492708 0.518056 0.885417 0.608333
15 0.501818 0.497268 0.974545 0.830601

['15', '0.501818', '0.497268', '0.974545', '0.830601']
2 data/obj/149115503.thumb.txt
0 0.501818 0.497268 0.974545 0.830601
15 0.498047 0.447917 0.855469 0.531250

['15', '0.498047', '0.447917', '0.855469', '0.531250']
3 data/obj/150175957.thumb.txt
0 0.498047 0.447917 0.855469 0.531250
15 0.471094 0.510417 0.814063 0.708333

['15', '0.471094', '0.510417', '0.814063', '0.708333']
4 data/obj/150121274.txt
0 0.471094 0.510417 0.814063 0.708333
15 0.516113 0.489676 0.893555 0.899705

['15', '0.516113', '0.489676', '0.893555', '0.899705']
5 data/obj/6912273093_8ef399a599_b.txt
0 0.516113 0.489676 0.893555 0.899705
15 0.475000 0.469167

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
images_list.extend(glob.glob("data/obj/*.jpeg"))
print(images_list)

['data/obj/149124216.jpg', 'data/obj/149109724.jpg', 'data/obj/149111530.jpg', 'data/obj/149108476.jpg', 'data/obj/150178738.jpg', 'data/obj/150084715.thumb.jpg', 'data/obj/149461591.jpg', 'data/obj/150082962.jpg', 'data/obj/149109321.jpg', 'data/obj/149152503.thumb.jpg', 'data/obj/149115218.jpg', 'data/obj/149150962.jpg', 'data/obj/149121231.jpg', 'data/obj/149115591.jpg', 'data/obj/150154102.jpg', 'data/obj/150150684.jpg', 'data/obj/149117072.thumb.jpg', 'data/obj/150201087.jpg', 'data/obj/150149815.jpg', 'data/obj/150198615.jpg', 'data/obj/150201292.jpg', 'data/obj/149129519.jpg', 'data/obj/150123221.jpg', 'data/obj/149116868.jpg', 'data/obj/150085089.jpg', 'data/obj/150200953.jpg', 'data/obj/149106309.thumb.jpg', 'data/obj/149150268.jpg', 'data/obj/149138229.jpg', 'data/obj/149126548.thumb.jpg', 'data/obj/149143430.jpg', 'data/obj/149147161.jpg', 'data/obj/149136355.jpg', 'data/obj/149109598.thumb.jpg', 'data/obj/149137573.jpg', 'data/obj/150175957.thumb.jpg', 'data/obj/150083324.j

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.630390, GIOU: 0.606227), Class: 0.824191, Obj: 0.016496, No Obj: 0.002894, .5R: 0.750000, .75R: 0.250000, count: 4, class_loss = 1.026198, iou_loss = 0.379939, total_loss = 1.406137 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000973, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.002055, iou_loss = 0.000000, total_loss = 0.002055 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000433, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.001794, iou_loss = 0.000000, total_loss = 0.001794 
 total_bbox = 25194, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.565332, GIOU: 0.544184), Class: 0.858759, Obj: 0.048372, No Obj: 

KeyboardInterrupt: ignored